In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
redis_conn.hkeys('algos')

['default']

In [267]:
######################
# Initiate Backtest  #
######################
from lib.multitasking_lib import *
cache_type ='backtest_uni'
my_cache = cache_state('backtest_uni')
#my_cache.flushall()
def backtest_unitest(stocks):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    algo = ""
    fromDate = '2019-5-2'
    toDate = '2019-5-3'
    freq = '1T'
    algo_name = 'haikin_1'
    backtest_msg={'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq}

    try:
        store_algo(algo, algo_name)
    except:
        perror('Something went wrong while saving algo')

    #pdebug1(backtest_msg)
    # Step 2: Store the stock name under backtest in the redis cache
    for stock in stocks:
        redis_conn.set('stock',stock) #TODO: replace

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
redis_conn.set('done'+cache_type,1)

try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler"], [kite_simulator, ohlc_tick_handler])
    
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


DEBUG:	Cache initialized for: backtest_uni
DEBUG:	Cache initialized for: backtest_uni
INFO:	Starting Handler: kite_simulator
INFO:	Starting Handler: ohlc_tick_handler


True
['tick_resamplerlive', 'ohlc_tick_handlerlive', 'ohlc_tick_handlerbacktest_web', 'order_handlerlive', 'kite_simulatorbacktest_web', 'freedom_initfreedom']
[<Process(Process-54, stopped[SIGTERM])>, <Process(Process-55, stopped[SIGTERM])>, <Process(Process-56, stopped[SIGTERM])>, <Process(Process-57, stopped[SIGTERM])>, <Process(Process-58, started)>]


DEBUG:	kite_simulator: 1
ERROR:	kite_simulator: Invalid msg: 1
DEBUG:	kite_simulator: {"stock": ["TCS"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1", "fromDate": "2019-5-2", "toDate": "2019-5-3", "freq": "1T"}
DEBUG:	Trade_init: TCS
INFO:	TCSbacktest_uni=>{'stock': 'TCS', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L %': '0.0', 'low': '0.0', 'sl': '0.0', 'ltp': '0.0', 'ltp %': '0.0', 'tp': '0.0', 'high': '0.0', 'last_processed': '1999-01-01', 'state': 'INIT', 'mode': 'PAUSE', 'algo': '', 'freq': '1D', 'hdf_freq': 'day'}
INFO:	Kite_Simulator: Done: 1500
TRADE: B: EN:  : 2019-05-02 09:34 : TCS -> 2239.0
TRADE: S: EX:  : 2019-05-02 09:52 : TCS -> 2239.45
TRADE: B: EN:  : 2019-05-02 09:59 : TCS -> 2236.9
TRADE: S: EX:  : 2019-05-02 10:19 : TCS -> 2249.9
TRADE: B: EN:  : 2019-05-02 10:31 : TCS -> 2238.05
TRADE: S: EX:  : 2019-05-02 10:50 : TCS -> 2233.9
TRADE: B: EN:  : 2019-05-02 11:05 : TCS -> 

In [268]:
backtest_unitest(['TCS'])

ERROR:	Something went wrong while saving algo


In [249]:
my_cache.delete('msgBufferQueue'+cache_postfix)
my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())

True

In [266]:
if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
backtest_manager.job.is_alive()
redis_conn.pubsub_channels()

['tick_resamplerlive',
 'ohlc_tick_handlerlive',
 'ohlc_tick_handlerbacktest_web',
 'order_handlerlive',
 'kite_simulatorbacktest_web',
 'freedom_initfreedom']

In [17]:
backtest_unitest(['TCS'])

In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [258]:
my_cache.getOHLC('TCS')

,close,high,low,open
2018-11-05,1890.85,1927.35,1884.75,1910.0
2018-11-06,1932.70,1937.50,1891.00,1894.0
2018-11-07,1942.15,1949.00,1937.00,1940.0
2018-11-09,1909.20,1961.95,1905.00,1961.0
2018-11-12,1914.85,1946.10,1908.50,1908.5
...,...,...,...,...
2019-04-16,2131.80,2155.95,2115.55,2120.0
2019-04-18,2150.05,2155.00,2114.20,2149.9
2019-04-22,2161.45,2176.00,2145.60,2148.0
2019-04-23,2155.05,2165.50,2134.05,2163.0


In [269]:
my_cache.getTrades('TCS')

,buy,mode,sell
2019-05-02 09:34:00,2239.00,B: EN:,NaN
2019-05-02 09:52:00,NaN,S: EX:,2239.45
2019-05-02 09:59:00,2236.90,B: EN:,NaN
2019-05-02 10:19:00,NaN,S: EX:,2249.90
2019-05-02 10:31:00,2238.05,B: EN:,NaN
2019-05-02 10:50:00,NaN,S: EX:,2233.90
2019-05-02 11:05:00,2228.45,B: EN:,NaN
2019-05-02 11:59:00,NaN,S: EX:,2220.00
2019-05-02 12:05:00,NaN,S: EN:,2219.95
2019-05-02 12:17:00,2215.60,B: EX:,NaN


In [270]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
fig = freedom_chart('TCS')
fig

''

In [263]:
# Terminate all the processes
backtest_manager.job.terminate()
time.sleep(0.1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
jobs
jobs = []

False
['tick_resamplerlive', 'ohlc_tick_handlerlive', 'ohlc_tick_handlerbacktest_web', 'order_handlerlive', 'kite_simulatorbacktest_web', 'freedom_initfreedom']


In [220]:
def resample(df, freq = '1T'):
    tmp_df = pd.DataFrame()    
    tmp_df = df.resample(freq,label='left').agg(['last','max','min','first']).dropna()
    tmp_df.columns = ['close', 'high', 'low', 'open']
    #print(tmp_df.head(5))
    return tmp_df
tmp_df = pd.read_json(my_cache.get('TCSbacktest_uniTICK'))
resample(tmp_df['ltp'],'1D')

,close,high,low,open
2018-11-05,1890.85,1927.35,1884.75,1910.00
2018-11-06,1932.70,1937.50,1891.00,1894.00
2018-11-07,1942.15,1949.00,1937.00,1940.00
2018-11-09,1909.20,1961.95,1905.00,1961.00
2018-11-12,1914.85,1946.10,1908.50,1908.50
...,...,...,...,...
2019-04-25,2195.00,2205.00,2168.05,2175.00
2019-04-26,2238.55,2243.95,2193.15,2203.00
2019-04-30,2260.35,2266.95,2230.10,2238.55
2019-05-02,2215.40,2259.40,2211.10,2255.00


In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

In [ ]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


## Redis Console

In [ ]:
from redis import Redis
import json

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

In [ ]:
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
# Unit Test#1
freq='day'
exchange='NSE'
fromDate = '2019-01-01'
toDate = '2019-05-03'
stock='INFY'

state = 'INIT'
hash_key = stock+'_state'

try:
    all_keys = list(conn.hgetall(hash_key).keys())
    conn.hdel(hash_key,*all_keys)
except:
    pass

#conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)


In [ ]:
# Unit Test#2
stock = 'NDTV'
msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

In [ ]:
conn.pubsub_channels()

In [ ]:
# Debug
stock='TCS'
#notification_despatcher(None, 'msg')
print(stock)
pd.read_json(conn.hget(stock+'_state','ohlc'))

In [ ]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

## Scratchpad

In [ ]:
redis_conn.pubsub_channels()

In [ ]:
my_cache = cache_state('backtest_web')

In [ ]:
df = my_cache.getValue()

In [ ]:
df

In [ ]:
trades_df = my_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [1]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [8]:
# GetData

logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [6]:
ohlc_data[stock].head()

,close,high,low,open,volume
date,,,,,
2019-04-22 09:15:00,285.90,286.50,285.15,285.50,167257
2019-04-22 09:16:00,286.45,286.55,285.60,285.85,128478
2019-04-22 09:17:00,286.40,286.55,286.00,286.25,105224
2019-04-22 09:18:00,286.80,287.00,286.35,286.40,96006
2019-04-22 09:19:00,287.65,287.80,286.80,286.80,100153


In [2]:
from lib.data_model_lib import *
live_cache = cache_state('backtest_web')
live_cache.getValue()
#live_cache.getOHLC('TCS', '1D').head()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,AXISBANK,10,1,1,7745.0,0,-3.9999999999997726,-0.5167291047667966,12.499999999995453,0.161394448030929,...,762.5,0.0,0,773.55,2019-04-01 15:29,SCANNING,PAUSE,default,minute,day
1,WIPRO,10,1,1,2898.5,0,1.4999999999997726,0.5172413793102664,3.999999999999204,0.1380024150422358,...,291.6,0.0,0,291.85,2019-04-23 15:29,SCANNING,PAUSE,default,1T,minute
2,TCS,10,1,1,21498.5,0,14.00000000000091,0.6516325723196217,-168.50000000000364,-0.7837756122520346,...,2163.0,0.0,0,2165.5,2019-04-23 15:29,SCANNING,PAUSE,default,1T,minute


In [9]:
msg_ltp = {
    "status": "success",
    "data": {
        "BSE:SENSEX": {
            "instrument_token": 265,
            "last_price": 31606.48
        },
        "NSE:INFY": {
            "instrument_token": 408065,
            "last_price": 890.9
        },
        "NSE:NIFTY 50": {
            "instrument_token": 256265,
            "last_price": 9893.4
        }
    }
}

In [94]:
def push_tick(ws, msg):
    live_cache.xadd('tickBuffer', {'data':json.dumps(msg_ltp['data'])})



In [153]:
push_tick(None, msg_ltp)

In [161]:
def tick_resampler():
    global tick_df
    #print(msg['data'])
    tickBuffer = live_cache.xread({'tickBuffer':'0-0'})
    tick_df = pd.DataFrame()
    for val in tickBuffer[0][1]:
        date_val = datetime.fromtimestamp(int(val[0].split('-')[0])/1000).strftime('%Y-%m-%d %H:%M:%S')
        stock = []
        ltp = []
        date = []
        for key, data in json.loads(val[1]['data']).items():
            stock_id = key.split(':')[1]
            stock.append(stock_id)
            ltp.append(json.loads(data)['last_price'])
            date.append(date_val)
        tmp_df = pd.DataFrame(data={'date':date,'stock':stock,'ltp':ltp})  
        tick_df = tick_df.append(tmp_df)
        
    return tick_df

#tick_resampler()



#df = tick_df[tick_df['stock']=='INFY'].set_index('date')
#df.index = pd.to_datetime(df.index)

df = tick_resampler()

df = df[df['stock']=='TCS'].set_index('date')
df.index = pd.to_datetime(df.index)
df.shape


(3000, 2)

In [191]:
def resample(df, freq = '1T'):
    tmp_df = pd.DataFrame()    
    tmp_df = df.resample(freq,label='left').agg(['last','max','min','first']).dropna()
    tmp_df.columns = ['close', 'high', 'low', 'open']
    #print(tmp_df.head(5))
    return tmp_df

#df['ltp'].shape
resample_df = resample(df['ltp'])

In [192]:
#live_cache.xlen('tickBuffer')

In [193]:
print(resample_df.shape)
resample_df.tail()

(750, 4)


,close,high,low,open
date,,,,
2019-04-23 15:25:00,2161.15,2161.60,2159.80,2160.75
2019-04-23 15:26:00,2160.00,2161.95,2159.30,2161.10
2019-04-23 15:27:00,2163.10,2165.30,2160.00,2160.00
2019-04-23 15:28:00,2164.75,2165.50,2163.15,2164.35
2019-04-23 15:29:00,2163.00,2164.90,2158.00,2164.00


In [185]:
ohlc_data['TCS'].tail()

,close,high,low,open,volume
date,,,,,
2019-04-23 15:25:00,2161.15,2161.60,2159.80,2160.75,9968
2019-04-23 15:26:00,2160.00,2161.95,2159.30,2161.10,11768
2019-04-23 15:27:00,2163.10,2165.30,2160.00,2160.00,52242
2019-04-23 15:28:00,2164.75,2165.50,2163.15,2164.35,13517
2019-04-23 15:29:00,2163.00,2164.90,2158.00,2164.00,6439


In [95]:
tickBuffer = live_cache.xread({'tickBuffer':'0-0'})

In [102]:
tickBuffer[0][1]

[('1588882591684-0',
  {'data': '{"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}'}),
 ('1588882594477-0',
  {'data': '{"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}'}),
 ('1588882596669-0',
  {'data': '{"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}'}),
 ('1588882598647-0',
  {'data': '{"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}'}),
 ('1588882600762-0',
  {'data': '{"BSE:SENSEX": {"instrument_tok

In [89]:
live_cache.xrange('tickBuffer')

[('1588882591684-0',
  {'data': '{"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}'}),
 ('1588882594477-0',
  {'data': '{"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}'}),
 ('1588882596669-0',
  {'data': '{"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}'}),
 ('1588882598647-0',
  {'data': '{"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}'}),
 ('1588882600762-0',
  {'data': '{"BSE:SENSEX": {"instrument_tok

In [ ]:
for stock,data in msg['data'].items():
        stock_name = stock.split(':')[1]
        ltp = data['last_price']
        print('{}=>{}'.format(stock_name, ltp))

In [ ]:
from lib.data_model_lib import *
live_cache = cache_state('backtest_web')
live_cache.getValue()

In [8]:
data = ['WIPRO','TCS']
fromDate = startDate = '2019-4-22'
toDate  = '2019-4-24'
hdf_freq = 'minute'
mydate = fromDate
exchange = 'NSE'

ohlc_data = {}
for stock_key in data: #Initialize
    # Load data from the Cache
    #cache.setValue(stock_key, 'hdf_freq', hdf_freq)        
    df = getData(stock_key, startDate, toDate, exchange, hdf_freq, False, stock_key)
    ohlc_data[stock_key] = df
    #trade_init(stock_key, algo, freq, qty, sl, target, hdf_freq)

#cache.publish('ohlc_tick_handler'+cache_postfix,'start')

#cache.set('logMsg'+cache_postfix,'Backtest Started: {} :\n'.format(stock)) # Used for displaying trade log



In [151]:
stock = data[-1]
no = ohlc_data[stock].shape[0]
#no = 2
counter = 0
live_cache.delete('tickBuffer')
stream_id = lambda x,y:str(int(x.timestamp()+y)*1000)+'-0'
for i in np.linspace(0,no-1,no): # Push data
    i = int(i)

    msg_dict_open = {}
    msg_dict_high = {}
    msg_dict_low = {}
    msg_dict_close = {}
    for stock in  data:
        row = ohlc_data[stock].iloc[i]
        index = ohlc_data[stock].index[i]
        
        #stream_id = str(int(index.timestamp())*1000)+'-0'
        msg = {exchange+":"+stock:json.dumps({"last_price":row['open']})}
        msg_dict_open.update(msg)
        
        #stream_id = str(int(index.timestamp())*1000)+'-0'
        msg = {exchange+":"+stock:json.dumps({"last_price":row['high']})}
        msg_dict_high.update(msg)
        
        #stream_id = str(int(index.timestamp())*1000)+'-0'
        msg = {exchange+":"+stock:json.dumps({"last_price":row['low']})}
        msg_dict_low.update(msg)
        
        #stream_id = str(int(index.timestamp())*1000)+'-0'
        msg = {exchange+":"+stock:json.dumps({"last_price":row['close']})}
        msg_dict_close.update(msg)
        
    
    live_cache.xadd('tickBuffer', {'data':json.dumps(msg_dict_open)}, id=stream_id(index,5))
    live_cache.xadd('tickBuffer', {'data':json.dumps(msg_dict_high)}, id=stream_id(index,10))
    live_cache.xadd('tickBuffer', {'data':json.dumps(msg_dict_low)}, id=stream_id(index,20))
    live_cache.xadd('tickBuffer', {'data':json.dumps(msg_dict_close)}, id=stream_id(index,30))
    
live_cache.xlen('tickBuffer')

3000

In [152]:
ohlc_data[stock].shape[0]

750

In [63]:
help(live_cache.xadd)

Help on method xadd in module redis.client:

xadd(name, fields, id='*', maxlen=None, approximate=True) method of lib.data_model_lib.cache_state instance
    Add to a stream.
    name: name of the stream
    fields: dict of field/value pairs to insert into the stream
    id: Location to insert this record. By default it is appended.
    maxlen: truncate old stream members beyond this size
    approximate: actual stream length may be slightly more than maxlen

